In [1]:
import json

from tqdm import tqdm

In [2]:
root = "/cache_ccks"
train_info_path = f"{root}/item_train_info.jsonl"
valid_info_path = f"{root}/item_valid_info.jsonl"
train_pair_path = f"{root}/item_train_pair.jsonl"
valid_pair_path = f"{root}/item_valid_pair.jsonl"

In [3]:
def read_info(json_path):
    items = [json.loads(line) for line in open(json_path).readlines()]
    ret = {}
    for item in items:
        item_id = item["item_id"]
        ret[item_id] = item
    return ret

In [4]:
def read_pair(json_path):
    items = [json.loads(line) for line in open(json_path).readlines()]
    return items

- `SKU`: Standard Product Unit （标准化产品单元），SPU是商品信息聚合的最小单位。

> [(产品 SKU 是什么意思？与之相关的还有哪些？ - kentzhu的回答 - 知乎](https://www.zhihu.com/question/19841574/answer/13126965)

In [5]:
import unicodedata


def count_p(info_paths):
    pv_dict = {}
    for info_path in info_paths:
        info = read_info(info_path)
        for _, item in info.items():
            for key in ["sku_pvs", "item_pvs"]:
                if key not in item:
                    continue
                pvs = item[key].split(";")
                for pv in pvs:
                    # if pv.count("#:") != 1:
                    #     print(pv.count("#:"), pv)
                    if pv.count("#:") == 0:
                        continue
                    pv_split = pv.split("#:")
                    p = unicodedata.normalize(
                        "NFKC", pv_split[0].replace("#", "").lower()
                    )
                    v = unicodedata.normalize(
                        "NFKC", "".join(pv_split[1:]).replace("#", "").lower()
                    )
                    if p not in pv_dict:
                        pv_dict[p] = set()
                    pv_dict[p].add(v)
    for p in pv_dict:
        pv_dict[p] = sorted(list(pv_dict[p]))
    return pv_dict

In [6]:
pv_dict = count_p([train_info_path, valid_info_path])
with open("pvs.json", "w") as f:
    f.write(json.dumps(pv_dict, ensure_ascii=False))
print(len(pv_dict))

1351


In [7]:
from collections import Counter

for p in pv_dict:
    pv_dict[p] = "".join(pv_dict[p])
chars = "".join(list(pv_dict.keys())) + "".join(list(pv_dict.values()))
print(len(list(set(chars))))
print(Counter(chars))

3947
Counter({'0': 118619, '1': 87418, '2': 75315, '5': 45069, '3': 41437, '6': 36392, '-': 34231, '4': 34083, '8': 33390, '9': 25927, 'm': 25332, '7': 24537, ' ': 19875, 'g': 19637, '色': 16681, '+': 15379, '【': 14622, 'c': 14353, '】': 14083, 'l': 13695, '(': 13190, 'a': 13002, '*': 12837, ')': 12832, 'x': 12561, '/': 11871, '.': 11861, 's': 11164, 'e': 9754, 'o': 9220, 'r': 8793, 'd': 8666, 'i': 8423, 'k': 8248, '款': 8198, 'p': 7987, 't': 7987, 'b': 7477, '黑': 7470, 'h': 7419, 'n': 7253, 'f': 6620, '装': 6444, 'y': 6112, '白': 6052, 'w': 6039, '加': 5233, '米': 4498, 'j': 4370, 'u': 4162, 'z': 4118, '妆': 4104, '水': 4060, '套': 3995, 'v': 3819, '网': 3633, '字': 3577, '电': 3537, '小': 3528, '送': 3489, '蓝': 3477, '层': 3450, '长': 3316, '一': 3295, '大': 3293, '高': 3283, '粉': 3221, '发': 3111, '机': 3044, '厚': 3020, '单': 2999, '备': 2954, 'q': 2899, '版': 2892, '手': 2852, '盒': 2788, '金': 2718, '包': 2702, '号': 2702, '头': 2680, '面': 2678, '斤': 2678, '_': 2585, '升': 2534, '级': 2520, '红': 2520, '品': 2383, 